In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_parquet('/Users/huangshifeng/Desktop/stage_III_colon_surv/data/prepare_ML.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Patient_ID            331 non-null    int64         
 1   Dx_Year               331 non-null    int64         
 2   Age                   331 non-null    int64         
 3   Sex                   331 non-null    int64         
 4   BMI                   327 non-null    float64       
 5   ECOG                  314 non-null    Int8          
 6   Tumor_Location        331 non-null    int64         
 7   Tumor_Location_Group  331 non-null    int64         
 8   pT_Stage              331 non-null    category      
 9   pN_Stage              331 non-null    category      
 10  AJCC_Substage         331 non-null    category      
 11  LN_Total              331 non-null    int64         
 12  LN_Positive           331 non-null    int64         
 13  LNR                 

In [9]:
#檢查FU時間夠不夠
df['Follow_Up_Time'] = round((((df['Last_FU_Date'] - df['Radical_Op_Date']).dt.days)/30), 1)
df['Follow_Up_Time'].head()

print("\n=== Follow-up Analysis ===")
print(f"Median FU: {df['Follow_Up_Time'].median():.1f} months")
print(f"Range: {df['Follow_Up_Time'].min():.1f} - {df['Follow_Up_Time'].max():.1f} months")
print(f"\nFU adequacy for 18m window:")
print(
    f"  >18 months: {(df['Follow_Up_Time']>18).sum()} / {len(df)} "
    f"({((df['Follow_Up_Time']>18).mean()*100):.1f}%)"
)
print(f"  <=18 months: {(df['Follow_Up_Time']<=18).sum()} (需進一步檢查)")

df_short_fu = df[df['Follow_Up_Time']<=18]
ids = df_short_fu['Patient_ID'].to_list()
print("")
print("追蹤時間<=18個月的病歷號：", ids)


#去掉那些沒復發也沒死的
df_short_fu.head()
df_no_events = df_short_fu[(df_short_fu['Death'] == 0) & (df_short_fu['Recurrence'] == 0)]
ids_for_confirm = df_no_events['Patient_ID'].to_list()
print()
print("這些病歷號要手動確認為何追蹤時間這麼短：", ids_for_confirm)


=== Follow-up Analysis ===
Median FU: 40.2 months
Range: 0.8 - 88.0 months

FU adequacy for 18m window:
  >18 months: 293 / 331 (88.5%)
  <=18 months: 38 (需進一步檢查)

追蹤時間<=18個月的病歷號： [5, 10, 12, 26, 28, 31, 32, 39, 46, 67, 69, 76, 80, 81, 83, 90, 96, 117, 119, 128, 130, 137, 139, 148, 154, 158, 178, 201, 217, 223, 238, 257, 275, 285, 295, 301, 304, 311]

這些病歷號要手動確認為何追蹤時間這麼短： []


In [10]:
#Dataset基本資訊
print("\n【BASIC INFO】")
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")


【BASIC INFO】
Total rows: 331
Total columns: 43


In [11]:
#Outcomes變數檢查
print("\n【TARGET VARIABLE CHECK-EDR-18m】")

# 檢查EDR-18m
if 'edr_18m' in df.columns:
    print(f"✓ Target variable found: edr_18m")
    print(f"  Data type: {df['edr_18m'].dtype}")
    print(f"  Value counts:")
    print(df['edr_18m'].value_counts().to_dict())
    print(f"  Event rate: {df['edr_18m'].mean()*100:.1f}%")
    print(f"  Events: {df['edr_18m'].sum()}")
    
    # 檢查是否只有0和1
    unique_vals = df['edr_18m'].unique()
    if set(unique_vals).issubset({0, 1}):
        print(f"  ✓ Binary values only: {sorted(unique_vals)}")
    else:
        print(f"  ✗ WARNING: Non-binary values found: {unique_vals}")
else:
    print("✗ ERROR: 'edr_18m' not found!")
    print(f"Available columns: {df.columns.tolist()}")

print()
print("\n【TARGET VARIABLE CHECK-EDR-24m】")
# 順便一起檢查EDR-24m
if 'edr_24m' in df.columns:
    print(f"✓ Target variable found: edr_24m")
    print(f"  Data type: {df['edr_24m'].dtype}")
    print(f"  Value counts:")
    print(df['edr_24m'].value_counts().to_dict())
    print(f"  Event rate: {df['edr_24m'].mean()*100:.1f}%")
    print(f"  Events: {df['edr_24m'].sum()}")
    
    # 檢查是否只有0和1
    unique_vals = df['edr_24m'].unique()
    if set(unique_vals).issubset({0, 1}):
        print(f"  ✓ Binary values only: {sorted(unique_vals)}")
    else:
        print(f"  ✗ WARNING: Non-binary values found: {unique_vals}")
else:
    print("✗ ERROR: 'edr_24m' not found!")
    print(f"Available columns: {df.columns.tolist()}")


【TARGET VARIABLE CHECK-EDR-18m】
✓ Target variable found: edr_18m
  Data type: Int8
  Value counts:
{np.int8(0): 269, np.int8(1): 62}
  Event rate: 18.7%
  Events: 62
  ✓ Binary values only: [np.int8(0), np.int8(1)]


【TARGET VARIABLE CHECK-EDR-24m】
✓ Target variable found: edr_24m
  Data type: Int8
  Value counts:
{np.int8(0): 263, np.int8(1): 68}
  Event rate: 20.5%
  Events: 68
  ✓ Binary values only: [np.int8(0), np.int8(1)]


In [12]:
print("\n【TRAIN/TEST SPLIT CHECK 拆分組檢查】")

if 'Op_Year' in df.columns:
    print("Year distribution:")
    year_dist = df['Op_Year'].value_counts().sort_index()
    for year, count in year_dist.items():
        events = df[df['Op_Year']==year]['edr_18m'].sum()
        print(f"  {year}: {count:3d} cases ({events:2d} events, {events/count*100:5.1f}%)")
    
    # 建議的split
    train = df[df['Op_Year'].between(2017, 2020)]
    test = df[df['Op_Year'] == 2021]
    
    print(f"\nSuggested split:")
    print(f"  Training (2017-2020): {len(train):3d} cases, {train['edr_18m'].sum():2d} events")
    print(f"  Testing (2021):       {len(test):3d} cases, {test['edr_18m'].sum():2d} events")
    
    # Events per variable check
    train_events = train['edr_18m'].sum()
    print(f"\n  Events-per-variable (EPV) in training:")
    for n_features in [10, 12, 15, 20]:
        epv = train_events / n_features
        status = "✓" if epv >= 5 else "⚠️"
        print(f"    {n_features} features: EPV = {epv:.1f} {status}")
else:
    print("✗ ERROR: 'Op_Year' not found!")


【TRAIN/TEST SPLIT CHECK 拆分組檢查】
Year distribution:
  2017:  44 cases (11 events,  25.0%)
  2018:  60 cases (10 events,  16.7%)
  2019:  75 cases (15 events,  20.0%)
  2020:  74 cases (13 events,  17.6%)
  2021:  78 cases (13 events,  16.7%)

Suggested split:
  Training (2017-2020): 253 cases, 49 events
  Testing (2021):        78 cases, 13 events

  Events-per-variable (EPV) in training:
    10 features: EPV = 4.9 ⚠️
    12 features: EPV = 4.1 ⚠️
    15 features: EPV = 3.3 ⚠️
    20 features: EPV = 2.5 ⚠️


In [13]:
print("\n【MISSING VALUES CHECK 缺失值檢查】")

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Pct': missing_pct
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Pct', ascending=False)

if len(missing_df) > 0:
    print(f"⚠️ Found {len(missing_df)} columns with missing values:")
    print(missing_df.head(20).to_string())
    
    # 高缺失率變數
    high_missing = missing_df[missing_df['Missing_Pct'] > 20]
    if len(high_missing) > 0:
        print(f"\n⚠️ HIGH MISSING (>20%):")
        print(high_missing.to_string())
else:
    print("✓ No missing values found!")


【MISSING VALUES CHECK 缺失值檢查】
⚠️ Found 14 columns with missing values:
                    Missing_Count  Missing_Pct
Recurrence_Date               242        73.11
Recurrence_Type               242        73.11
time_to_recurrence            242        73.11
CEA_PreOp                      81        24.47
Log_CEA_PreOp                  81        24.47
PreOp_Albumin                  53        16.01
ECOG                           17         5.14
BMI                             4         1.21
LVI                             4         1.21
PNI                             3         0.91
MSI_Status                      2         0.60
Tumor_Size_cm                   2         0.60
DFS_Months                      2         0.60
OS_Months                       2         0.60

⚠️ HIGH MISSING (>20%):
                    Missing_Count  Missing_Pct
Recurrence_Date               242        73.11
Recurrence_Type               242        73.11
time_to_recurrence            242        73.11
CEA_PreOp  

In [22]:
print("\n【DATA TYPES CHECK 資料型態檢查】")

dtypes_summary = df.dtypes.value_counts()
print("Data type distribution:")
for dtype, count in dtypes_summary.items():
    print(f"  {dtype}: {count} columns")

# 檢查object型態（可能需要編碼）
object_cols = df.select_dtypes(include=['object']).columns.tolist()
if len(object_cols) > 0:
    print(f"\n⚠️ Object columns found (may need encoding): {len(object_cols)}")
    for col in object_cols[:10]:  # 只顯示前10個
        unique = df[col].nunique()
        print(f"  {col}: {unique} unique values")
        if unique <= 10:
            print(f"    → {df[col].value_counts().head(5).to_dict()}")


【DATA TYPES CHECK 資料型態檢查】
Data type distribution:
  int64: 17 columns
  float64: 12 columns
  Int8: 4 columns
  datetime64[ns]: 3 columns
  category: 1 columns
  category: 1 columns
  category: 1 columns
  category: 1 columns
  category: 1 columns
  category: 1 columns
  int32: 1 columns


In [15]:
#上個步驟LNR的dtype看起來有問題，立馬處理

df['LNR_clean'] = df['LNR'].str.strip().str.lower()
df['LNR_clean'] = df['LNR_clean'].str.replace("%", "")
df['LNR'] = round(df['LNR_clean'].astype(float)/100, 3)
df['LNR'].head()
df = df.drop(columns="LNR_clean")

#重新確認一次
df.dtypes

Patient_ID                       int64
Dx_Year                          int64
Age                              int64
Sex                              int64
BMI                            float64
ECOG                              Int8
Tumor_Location                   int64
Tumor_Location_Group             int64
pT_Stage                      category
pN_Stage                      category
AJCC_Substage                 category
LN_Total                         int64
LN_Positive                      int64
LNR                            float64
Histology                        int64
Differentiation                  int64
LVI                            float64
PNI                            float64
Tumor_Deposits                   int64
Mucinous_Gt_50                   int64
Mucinous_Any                     int64
Signet_Ring                      int64
MSI_Status                    category
Tumor_Size_cm                  float64
CEA_PreOp                      float64
Log_CEA_PreOp            

In [16]:
df['LNR'].head()

0    0.706
1    0.059
2    0.000
3    0.176
4    0.167
Name: LNR, dtype: float64

In [17]:
print("\n【CATEGORICAL FEATURES CHECK 類別變數檢查】")

categorical_candidates = ['AJCC_Substage', 'pT', 'pN', 'Differentiation', 
                         'Tumor_Location', 'LVI', 'PNI', 'Tumor_Deposits']

for col in categorical_candidates:
    if col in df.columns:
        n_unique = df[col].nunique()
        print(f"\n  {col}:")
        print(f"    Unique values: {n_unique}")
        if n_unique <= 20:
            vc = df[col].value_counts()
            print(f"    Distribution: {vc.to_dict()}")
            
            # 檢查稀有類別
            rare = vc[vc < len(df) * 0.01]  # <1%的類別
            if len(rare) > 0:
                print(f"    ⚠️ Rare categories (<1%): {rare.index.tolist()}")


【CATEGORICAL FEATURES CHECK 類別變數檢查】

  AJCC_Substage:
    Unique values: 3
    Distribution: {'3B': 227, '3C': 67, '3A': 37}

  Differentiation:
    Unique values: 5
    Distribution: {2: 299, 3: 23, 4: 5, 1: 3, 9: 1}
    ⚠️ Rare categories (<1%): [1, 9]

  Tumor_Location:
    Unique values: 8
    Distribution: {7: 133, 2: 64, 8: 64, 1: 21, 4: 20, 6: 19, 3: 7, 5: 3}
    ⚠️ Rare categories (<1%): [5]

  LVI:
    Unique values: 2
    Distribution: {1.0: 167, 0.0: 160}

  PNI:
    Unique values: 2
    Distribution: {0.0: 274, 1.0: 54}

  Tumor_Deposits:
    Unique values: 2
    Distribution: {0: 314, 1: 17}


In [18]:
#處理一下Differentiation
df['Differentiation'] = df['Differentiation'].replace(9, np.nan).astype("Int8")
df['Differentiation'].value_counts()
df['Differentiation'].isna().sum()

np.int64(1)

In [19]:
print("\n【9. DUPLICATE CHECK】")

duplicates = df.duplicated().sum()
if duplicates > 0:
    print(f"⚠️ WARNING: Found {duplicates} duplicate rows")
else:
    print("✓ No duplicate rows")


【9. DUPLICATE CHECK】
✓ No duplicate rows


In [20]:
#拆分訓練級和驗證集
train = df[df['Op_Year'].between(2017, 2020)].copy()
test = df[df['Op_Year'] == 2021].copy()

In [21]:
train.to_parquet('/Users/huangshifeng/Desktop/stage_III_colon_surv/data/train.parquet')
test.to_parquet('/Users/huangshifeng/Desktop/stage_III_colon_surv/data/test.parquet')